# Mount Google Drive

In [0]:
!zip -r all_output_images.zip output

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Install OpenCV

In [2]:
!pip install opencv-python

## Define Base Directory

In [0]:
BASE_DIR = "/content/"

## Make Initial Project Directory

In [12]:
!mkdir input
!mkdir input/src
!mkdir input/dest
!mkdir output
!mkdir output/src
!mkdir output/dest
!mkdir output/mask
!mkdir output/final
!mkdir ml

mkdir: cannot create directory ‘input’: File exists
mkdir: cannot create directory ‘input/src’: File exists
mkdir: cannot create directory ‘input/dest’: File exists
mkdir: cannot create directory ‘ml’: File exists


# 0.1 Copy Source Video From Drive

In [0]:
!cp "/content/drive/My Drive/Final_Projects/ML/Input/src_1.mp4" /content/input/src/src_1.mp4

# 0.2 Copy Destination Video from Drive

In [0]:
!cp "/content/drive/My Drive/Final_Projects/ML/Input/dest_1.mp4" /content/input/dest/dest_1.mp4

# 1.1 Source Video to Image Convertion

In [13]:
import cv2

property_id = int(cv2.CAP_PROP_FRAME_COUNT)
print(property_id)
cap1 = cv2.VideoCapture(BASE_DIR+"input/src/src_1.mp4")
src_len = int(cv2.VideoCapture.get(cap1, property_id))
print("No Of Frames:" , src_len)

success,image = cap1.read()
count = 0;
while success:
    success,image = cap1.read()
    if image is not None:
        resized_image = cv2.resize(image,(720, 720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)   #resize image
        cv2.imwrite(BASE_DIR+"output/src/frame_%d.jpg" % count, resized_image)     # save frame as JPEG file
        if count == src_len-2:
            break
        count += 1

print("Source Images Saved")

7
No Of Frames: 456
Source Images Saved


# 1.2 Destination Video to Image Convertion

In [18]:
property_id = int(cv2.CAP_PROP_FRAME_COUNT)
print(property_id)
cap1 = cv2.VideoCapture(BASE_DIR+"input/dest/dest_1.mp4")
dest_len = int(cv2.VideoCapture.get(cap1, property_id))
print("No Of Frames:" , dest_len)

success,image = cap1.read()
count = 0;
while success:
    success,image = cap1.read()
    if image is not None:
        resized_image = cv2.resize(image,(720, 720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)   #resize image
        cv2.imwrite(BASE_DIR+"output/dest/frame_%d.jpg" % count, resized_image)     # save frame as JPEG file
        if cv2.waitKey(10) == 27:                     # exit if Escape is hit
            break
        if count == dest_len-2:
            break
        count += 1

print("Dest Images Saved")

7
No Of Frames: 453
Dest Images Saved


# 1.3 Define NO_OF_FRAMES To Process

In [0]:
NO_OF_FRAMES = min(src_len,dest_len) - 2
NO_OF_FRAMES

# 2.0 Install ML Releated Libs

In [20]:
!pip install keras tensorflow numpy matplotlib pillow

# 2.1 Add Source Code Directory to System Path

In [21]:
import sys
sys.path.append("/content/ml/")
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/ml/']

# 2.2 Copy Deeplab Model Codes to ML Directory

In [0]:
!cp -r "/content/drive/My Drive/Final_Projects/ML/Codes/extract_weights.py" /content/ml/
!cp -r "/content/drive/My Drive/Final_Projects/ML/Codes/load_weights.py" /content/ml/
!cp -r "/content/drive/My Drive/Final_Projects/ML/Codes/model.py" /content/ml/

# 2.3 Import ML Related Libs

In [23]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from model import Deeplabv3 as DL
import glob
import cv2

# 2.4 Initialize DeeplabV3 Model with Input Sizes

In [24]:
deeplab_model = DL(input_shape=(720, 720, 3))
deeplab_model

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
166100992/166096424 [==============================] - 12s 0us/step


# 2.5 Make List of Files to Predict Semantic Segmentation

In [27]:
src_imgs = []
for i in range(NO_OF_FRAMES):
    filename = "output/src/frame_%d.jpg"%i
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    src_imgs.append(img)

src_imgs = np.array(src_imgs).astype('uint8')
mean_subtraction_value=127.5
src_imgs = (src_imgs / mean_subtraction_value) - 1.
print('src_imgs shape:', src_imgs.shape)

src_imgs shape: (10, 720, 720, 3)


# 2.6 Make Filter for Cat Prediction

In [0]:
def apply_filter(inp_mat, x):
    inp_mat[inp_mat<x] *= 0
    inp_mat[inp_mat>x] *= 0
    inp_mat[inp_mat>0] = 255
    return inp_mat

# 2.7 Predict Semantic Segmentation and Save to mask Directory

In [29]:
number = 0
for i in range(0,NO_OF_FRAMES):
    res = deeplab_model.predict(np.expand_dims(src_imgs[i], 0))
    labels = np.argmax(res.squeeze(), -1)
    labels = apply_filter(labels, 8)
    filename = BASE_DIR + "output/mask/frame_%d.jpg"%i
    cv2.imwrite(filename, labels)
    res = None
    labels = None
    print("Frame %d Prediction and Saving Done"%i)

Frame 0 Prediction and Saving Done
Frame 1 Prediction and Saving Done
Frame 2 Prediction and Saving Done
Frame 3 Prediction and Saving Done
Frame 4 Prediction and Saving Done
Frame 5 Prediction and Saving Done
Frame 6 Prediction and Saving Done
Frame 7 Prediction and Saving Done
Frame 8 Prediction and Saving Done
Frame 9 Prediction and Saving Done


In [0]:
# from google.colab.patches import cv2_imshow

In [0]:
# from matplotlib import pyplot as plt
# plt.imshow(resized_image)

# 3.1 Define Mask Center

In [30]:
center = (720//2, 720//2)
center

(360, 360)

### 3.2 Apply Poisson Image Blending from Source to Dest. Based on MASK

In [31]:
for i in range(NO_OF_FRAMES):
    src = cv2.imread("output/src/frame_%d.jpg"%i, cv2.IMREAD_UNCHANGED)
    dst = cv2.imread("output/dest/frame_%d.jpg"%i, cv2.IMREAD_UNCHANGED)
    src_mask = cv2.imread("output/mask/frame_%d.jpg"%i, cv2.IMREAD_UNCHANGED)
    output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)
    cv2.imwrite(BASE_DIR + "output/final/frame_%d.jpg"%i, output)
    print("Frame %d Blending Done"%i)

Frame 0 Blending Done
Frame 1 Blending Done
Frame 2 Blending Done
Frame 3 Blending Done
Frame 4 Blending Done
Frame 5 Blending Done
Frame 6 Blending Done
Frame 7 Blending Done
Frame 8 Blending Done
Frame 9 Blending Done


# 3.3 Saving All Frames to a Video

In [0]:
img_array = []
for i in range(NO_OF_FRAMES):
    filename = "output/final/frame_%d.jpg"%i
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('output/output.avi', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(NO_OF_FRAMES):
    out.write(img_array[i])
    print("Frame %d Writing to Video Done"%i)
out.release()